In [ ]:
sheetid = '11BDaYyM7hnbbk8eLmboYKyLauvycyApCu38m3mu3ySM' # paste ur sheet id here
sheet_tab_name_where_data_tobe_inerted='Copy of Punch_Log'
sheet_tab_name_where_PIVOT_Table_tobe_inerted='Test2'



    # ******* Lookups ******

shifts={'03:00:00'	:'0_Early_Shift',
        '06:00:00':	'1_First_Shift_In' ,
        '09:00:00':	'2_Second_Shift_In',
        '12:00:00'	:'3_Third_Shift_In',
        '15:00:00':	'4_Break_Out',
        '18:00:00':	'5_Break_In',
        '21:00:00':	'6_First_Shift_Out',
        '00:00:00':	'7_Second_Shift_Out'}


Punch_Month_Number ={"Apr"	:	"01_April",
                    "May"	:	"02_May",
                    "Jun"	:	"03_June",
                    "Jul"	:	"04_July",
                    "Aug"	:	"05_August",
                    "Sep"	:	"06_September",
                    "Oct"	:	"07_October",
                    "Nov"	:	"08_November",
                    "Dec"	:	"09_December",
                    "Jan"	:	"10_January",
                    "Feb"	:	"11_February",
                    "Mar"	:	"12_March"}


# ******************************************************************* MAIN CODE ***********************************************************************

#@title RUN Here By CLICK HERE
# Import the 'auth' module from the 'google.colab' package
from google.colab import auth
from google.auth import default# Import the 'default' module from the 'google.auth' package
from pydrive.auth import GoogleAuth # Import the 'GoogleAuth' class from the 'pydrive.auth' module

from pydrive.drive import GoogleDrive # Import the 'GoogleDrive' class from the 'pydrive.drive' module
import gspread # Import the 'gspread' library for accessing Google Sheets

auth.authenticate_user() # Authenticate the user in the Colab environment

# Retrieve the default credentials for the authenticated user
creds, _ = default()

# Authorize the 'gspread' library with the obtained credentials
gc = gspread.authorize(creds)


# importing libraries
import pandas as pd 
import shutil
import os
import datetime 
import numpy as np
#
import re



import gspread_dataframe as gd

ws = gc.open_by_key(f'{sheetid}') # open sheet by key
worksheet=ws.worksheet(f'{sheet_tab_name_where_data_tobe_inerted}')

rows=worksheet.get_all_values()
df=pd.DataFrame.from_records(rows[1:],columns=rows[0]) #assiging as dataframe



# ******************************************************************* MAIN CODE ***********************************************************************
# Convert 'Punch Time' column to datetime format, assuming the time format is '%H:%M:%S'
df['Punch Time'] = pd.to_datetime(df['Punch Time'], format='%H:%M:%S')


# Convert 'Punch Date' column to datetime format, assuming the date format is '%d-%b-%Y'
df['Punch Date'] = pd.to_datetime(df['Punch Date'], format="%d-%b-%Y")
df['Punch Date'] = df['Punch Date'].dt.strftime('%d-%b-%Y')


# Extract the time component from 'Punch Time' column
df['Punch Time'] = df['Punch Time'].dt.time


# Extract the year from 'Punch Date' column and create a new column 'Punch_year'
df['Punch_year'] = df['Punch Date'].str[-4:]
df['Punch_month'] = df['Punch Date'].str.split('-').str[1]


# Map the month abbreviation to the corresponding month number and create a new column 'Punch_Month_Number'
df['Punch_Month_Number'] = df['Punch_month'].map(Punch_Month_Number)
# Round the 'Punch Time' to the nearest 3-hour interval and create a new column 'Interval'
df['Interval'] = df['Punch Time'].apply(lambda x: x.replace(hour=(x.hour // 3) * 3, minute=0, second=0))
df['Interval'] = df['Interval'].astype(str)


df['Shift'] = df['Interval'].map(shifts)

# If 'Shift' is '7_Second_Shift_Out', subtract 1 day from 'Punch Date' and assign it to
# 'Shift_Date', else assign 'Punch Date'
df['Shift_Date'] = np.where(df['Shift'] == '7_Second_Shift_Out', pd.to_datetime(df['Punch Date']) - pd.DateOffset(1), df['Punch Date'])
df['Shift_Date'] = pd.to_datetime(df['Shift_Date'])



df['Shift_Date'] = df['Shift_Date'].dt.strftime('%d-%b-%Y')

# Extract the month abbreviation from 'Shift_Date' column and create a new column 'Shift_Month'
df['Shift_Month'] = df['Shift_Date'].str.split('-').str[1]
df['Shift_Month_Number'] = df['Shift_Month'].map(Punch_Month_Number)


# Check if 'Punch Date' is equal to 'Shift_Date' and create a new column 'Is_Punch_Date_equalto_Shift_Date' with values 'Yes' or 'No'
df['Is_Punch_Date_equalto_Shift_Date'] = np.where(df['Punch Date'] == df['Shift_Date'], "Yes", "No")


df = df[['No', 'Mchn', 'EnNo', 'Name', 'Mode', 'NewLOG', 'Punch Date', 'Punch Time', 'Punch_year', 'Punch_month', 'Punch_Month_Number', 'Interval', 'Shift', 'Shift_Date', 'Shift_Month', 'Shift_Month_Number', 'Is_Punch_Date_equalto_Shift_Date']]

# Create a pivot table from the dataframe based on 'Name', 'Shift_Month_Number', and 'Shift_Date' as indices, 'Shift' as columns, and 'Punch Time' as values
pivoted_df = df.pivot_table(index=['Name', 'Shift_Month_Number', 'Shift_Date'], columns='Shift', values='Punch Time', aggfunc='first')


# Convert the pivot table to a DataFrame
df_pivoted = pd.DataFrame(pivoted_df)
df_pivoted = df_pivoted.reset_index()


# Create a new DataFrame with selected columns from the original DataFrame
new_df = df[['Punch_year', 'Punch_month', 'Punch_Month_Number', 'Interval', 'Shift', 'Shift_Date', 'Shift_Month', 'Shift_Month_Number', 'Is_Punch_Date_equalto_Shift_Date']]

# Get a range of cells in worksheet2 to update based on the shape of new_df
worksheet2 = ws.worksheet(f'{sheet_tab_name_where_data_tobe_inerted}')
cell_list = worksheet2.range("I2:Q{}".format(new_df.shape[0] + 1))



# ************************************************************* Code******************************************************************************
cell_values = new_df.values.tolist()

for i, val in enumerate(cell_values):
    for j in range(len(val)):
        cell_list[i*len(val) + j].value = val[j]

worksheet2.update_cells(cell_list)


worksheet3 = ws.worksheet(f'{sheet_tab_name_where_PIVOT_Table_tobe_inerted}')

import gspread_dataframe as gd
gd.set_with_dataframe(worksheet3,df_pivoted)